# 02 — ML Modeling: Ridge, LightGBM, Prophet
## HVAC Market Analysis — Metropolitan France (96 departments)

**Objective**: Train and compare 3 ML models to predict heat pump installations.

**Models**:
- **Ridge Regression** (Tier 1) — Robust baseline, L2-regularized linear regression
- **LightGBM** (Tier 2) — Gradient boosting, captures non-linearities
- **Prophet** (Tier 1) — Time series with external regressors

**Temporal split**:
- Train: 2021-07 -> 2024-06
- Validation: 2024-07 -> 2024-12
- Test: 2025-01 -> 2025-12

**Target variable**: `nb_installations_pac` (DPE mentioning a heat pump)

In [ ]:
# ============================================================
# IMPORTS
# ============================================================
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
import lightgbm as lgb

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

from config.settings import config
print('Imports OK')

---
## 1. Data loading and preparation

In [ ]:
# ============================================================
# 1.1 — Load the engineered features dataset
# ============================================================
# This dataset contains ~90+ columns: base features + lags + rolling + interactions
# + SITADEL (construction) + INSEE Filosofi (socioeconomic reference)

df = pd.read_csv('../data/features/hvac_features_dataset.csv')
print(f'Dataset: {df.shape[0]} rows x {df.shape[1]} columns')
print(f'Period: {df["date_id"].min()} -> {df["date_id"].max()}')
print(f'Departments: {df["dept"].nunique()}')
df.head(3)

In [ ]:
# ============================================================
# 1.2 — Define target variable and temporal split
# ============================================================
TARGET = 'nb_installations_pac'

# Split dates (YYYYMM format)
TRAIN_END = 202406   # Last training date
VAL_END = 202412     # Last validation date

# Temporal split (respects chronology -> no data leakage)
df_train = df[df['date_id'] <= TRAIN_END].copy()
df_val = df[(df['date_id'] > TRAIN_END) & (df['date_id'] <= VAL_END)].copy()
df_test = df[df['date_id'] > VAL_END].copy()

print(f'Train: {len(df_train)} rows ({df_train["date_id"].min()} -> {df_train["date_id"].max()})')
print(f'Val:   {len(df_val)} rows ({df_val["date_id"].min()} -> {df_val["date_id"].max()})')
print(f'Test:  {len(df_test)} rows ({df_test["date_id"].min()} -> {df_test["date_id"].max()})')

In [ ]:
# ============================================================
# 1.3 — Separate features (X) and target (y)
# ============================================================
# Columns to exclude from training (identifiers, metadata, other targets)
# Outlier flags are also excluded to prevent data leakage

EXCLUDE_COLS = {
    'date_id', 'dept', 'dept_name', 'city_ref', 'latitude', 'longitude',
    'n_valid_features', 'pct_valid_features',
    # Other targets (we predict nb_installations_pac)
    'nb_installations_clim', 'nb_dpe_total', 'nb_dpe_classe_ab',
    'pct_pac', 'pct_clim', 'pct_classe_ab',
}

OUTLIER_PATTERNS = ['_outlier_iqr', '_outlier_zscore', '_outlier_iforest',
                    '_outlier_consensus', '_outlier_score']

feature_cols = [
    c for c in df.columns
    if c not in EXCLUDE_COLS and c != TARGET
    and not any(p in c for p in OUTLIER_PATTERNS)
]
# Keep only numeric columns
feature_cols = [c for c in feature_cols if df[c].dtype in [np.float64, np.int64, np.float32, np.int32]]

X_train, y_train = df_train[feature_cols], df_train[TARGET]
X_val, y_val = df_val[feature_cols], df_val[TARGET]
X_test, y_test = df_test[feature_cols], df_test[TARGET]

print(f'Selected features: {len(feature_cols)}')
print(f'X_train: {X_train.shape}, y_train: {y_train.shape}')

In [ ]:
# ============================================================
# 1.4 — NaN imputation and standardization
# ============================================================
# NaN come from lags (start of series) and rolling windows
# Strategy: median imputation (robust to outliers)

# Drop all-NaN columns before imputation (SimpleImputer silently drops them,
# causing shape mismatch when rebuilding the DataFrame)
all_nan_cols = [c for c in feature_cols if X_train[c].isna().all()]
if all_nan_cols:
    print(f'Dropping {len(all_nan_cols)} all-NaN columns: {all_nan_cols}')
    feature_cols = [c for c in feature_cols if c not in all_nan_cols]
    X_train = df_train[feature_cols]
    X_val = df_val[feature_cols]
    X_test = df_test[feature_cols]

imputer = SimpleImputer(strategy='median')
X_train_imp = pd.DataFrame(
    imputer.fit_transform(X_train), columns=feature_cols, index=X_train.index
)
X_val_imp = pd.DataFrame(
    imputer.transform(X_val), columns=feature_cols, index=X_val.index
)
X_test_imp = pd.DataFrame(
    imputer.transform(X_test), columns=feature_cols, index=X_test.index
)

# Standardization (for Ridge — tree-based models don't need it)
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train_imp), columns=feature_cols, index=X_train.index
)
X_val_scaled = pd.DataFrame(
    scaler.transform(X_val_imp), columns=feature_cols, index=X_val.index
)
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test_imp), columns=feature_cols, index=X_test.index
)

print(f'Features after NaN column removal: {len(feature_cols)}')
print(f'NaN after imputation: {X_train_imp.isna().sum().sum()}')
print(f'Data ready for training!')

---
## 2. Ridge Regression (Baseline)

**Why Ridge?**
- L2 regularization -> stable even with correlated features (lags, rolling)
- Very robust on small-to-medium datasets
- Interpretable: coefficients indicate each feature's impact

We select the best alpha via temporal cross-validation.

In [ ]:
# ============================================================
# 2.1 — Hyperparameter selection (alpha) via temporal CV
# ============================================================
# TimeSeriesSplit respects chronology (no leakage)

alphas = [0.01, 0.1, 0.5, 1.0, 5.0, 10.0, 50.0, 100.0]
tscv = TimeSeriesSplit(n_splits=3)

results_alpha = []
for alpha in alphas:
    rmses = []
    for train_idx, val_idx in tscv.split(X_train_scaled):
        model = Ridge(alpha=alpha)
        model.fit(X_train_scaled.iloc[train_idx], y_train.iloc[train_idx])
        y_pred = model.predict(X_train_scaled.iloc[val_idx])
        rmse = np.sqrt(mean_squared_error(y_train.iloc[val_idx], y_pred))
        rmses.append(rmse)
    results_alpha.append({
        'alpha': alpha,
        'rmse_mean': np.mean(rmses),
        'rmse_std': np.std(rmses),
    })

df_alpha = pd.DataFrame(results_alpha)
best_alpha = df_alpha.loc[df_alpha['rmse_mean'].idxmin(), 'alpha']

# Visualize
fig, ax = plt.subplots(figsize=(10, 5))
ax.errorbar(df_alpha['alpha'], df_alpha['rmse_mean'], 
            yerr=df_alpha['rmse_std'], fmt='-o', capsize=5)
ax.axvline(best_alpha, color='red', linestyle='--', label=f'Best alpha = {best_alpha}')
ax.set_xscale('log')
ax.set_xlabel('Alpha (log scale)')
ax.set_ylabel('RMSE (temporal CV)')
ax.set_title('Ridge — Alpha selection via temporal cross-validation')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

print(f'\nBest alpha: {best_alpha} (CV RMSE = {df_alpha["rmse_mean"].min():.2f})')

In [ ]:
# ============================================================
# 2.2 — Final Ridge training
# ============================================================
ridge_model = Ridge(alpha=best_alpha)
ridge_model.fit(X_train_scaled, y_train)

# Predictions (clipped to 0 — no negative counts)
y_pred_val_ridge = np.clip(ridge_model.predict(X_val_scaled), 0, None)
y_pred_test_ridge = np.clip(ridge_model.predict(X_test_scaled), 0, None)

# Metrics
print('RIDGE REGRESSION')
print('=' * 50)
for name, y_true, y_pred in [('Validation', y_val, y_pred_val_ridge), 
                               ('Test', y_test, y_pred_test_ridge)]:
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f'  {name:12s} : RMSE={rmse:.2f}, MAE={mae:.2f}, R2={r2:.4f}')

In [ ]:
# ============================================================
# 2.3 — Ridge feature importance (absolute coefficients)
# ============================================================
importance_ridge = pd.Series(
    np.abs(ridge_model.coef_), index=feature_cols
).sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(10, 8))
importance_ridge.head(20).iloc[::-1].plot(kind='barh', ax=ax, color='steelblue')
ax.set_title('Ridge — Top 20 Features (|coefficient|)', fontsize=14)
ax.set_xlabel('Importance (|coef|)')
ax.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

---
## 3. LightGBM (Gradient Boosting)

**Why LightGBM?**
- Captures non-linear interactions between features
- Natively handles NaN (no explicit imputation needed)
- Strong regularization to avoid overfitting (max_depth=4, num_leaves=15)
- Early stopping on validation

In [ ]:
# ============================================================
# 3.1 — LightGBM training with early stopping
# ============================================================
# Constrained hyperparameters for moderate dataset size
lgb_params = {
    'max_depth': 4,              # Shallow trees
    'num_leaves': 15,            # Few leaves
    'min_child_samples': 20,     # Well-populated leaves
    'reg_alpha': 0.1,            # L1 regularization
    'reg_lambda': 0.1,           # L2 regularization
    'learning_rate': 0.05,       # Slow learning
    'n_estimators': 200,         # Max 200 trees
    'subsample': 0.8,            # Bagging
    'verbose': -1,
    'random_state': 42,
}

lgb_model = lgb.LGBMRegressor(**lgb_params)

# Training with early stopping (stop if val doesn't improve)
# Record both train and val metrics to plot learning curves
lgb_model.fit(
    X_train_imp, y_train,
    eval_set=[(X_train_imp, y_train), (X_val_imp, y_val)],
    eval_names=['train', 'validation'],
    eval_metric='rmse',
    callbacks=[
        lgb.early_stopping(stopping_rounds=20, verbose=False),
        lgb.log_evaluation(period=0),
    ],
)

# Predictions
y_pred_val_lgb = np.clip(lgb_model.predict(X_val_imp), 0, None)
y_pred_test_lgb = np.clip(lgb_model.predict(X_test_imp), 0, None)

# Metrics
print(f'LightGBM — Best iteration: {lgb_model.best_iteration_} / 200')
print('=' * 50)
for name, y_true, y_pred in [('Validation', y_val, y_pred_val_lgb), 
                               ('Test', y_test, y_pred_test_lgb)]:
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f'  {name:12s} : RMSE={rmse:.2f}, MAE={mae:.2f}, R2={r2:.4f}')

In [ ]:
# ============================================================
# 3.1a — LightGBM Learning Curve (overfitting diagnostic)
# ============================================================
# The learning curve shows train vs validation error over boosting rounds.
# Divergence between the two curves indicates overfitting.
# Early stopping prevents the model from using rounds past the optimal point.

evals_result = lgb_model.evals_result_

fig, ax = plt.subplots(figsize=(12, 5))

train_rmse = evals_result['train']['rmse']
val_rmse = evals_result['validation']['rmse']
rounds = range(1, len(train_rmse) + 1)

ax.plot(rounds, train_rmse, label='Train RMSE', linewidth=2, color='steelblue')
ax.plot(rounds, val_rmse, label='Validation RMSE', linewidth=2, color='darkorange')

# Mark the best iteration (early stopping point)
best_iter = lgb_model.best_iteration_
ax.axvline(best_iter, color='red', linestyle='--', alpha=0.7,
           label=f'Best iteration = {best_iter}')

# Shade the "overfitting zone" (after best iteration)
if best_iter < len(train_rmse):
    ax.axvspan(best_iter, len(train_rmse), alpha=0.1, color='red',
               label='Overfitting zone (stopped by early stopping)')

ax.set_xlabel('Boosting round')
ax.set_ylabel('RMSE')
ax.set_title('LightGBM — Learning Curve (Train vs Validation)', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Overfitting gap at best iteration
# NOTE: The RMSE ratio can appear large when train RMSE is very small (near-perfect
# fit on training data is normal for gradient boosting with early stopping).
# The definitive overfitting diagnostic uses R² gap (section 5.4 below).
train_best = train_rmse[best_iter - 1]
val_best = val_rmse[best_iter - 1]
gap_pct = (val_best - train_best) / train_best * 100
print(f'At best iteration ({best_iter}):')
print(f'  Train RMSE: {train_best:.2f}')
print(f'  Val RMSE:   {val_best:.2f}')
print(f'  RMSE ratio gap: {gap_pct:.1f}%')
print(f'  NOTE: High RMSE ratio is expected for tree ensembles (near-zero train error).')
print(f'  The definitive overfitting test is the R² gap in section 5.4 below.')

### 3.1b — LightGBM Hyperparameter Optimization

The parameters above were set manually. To ensure we're not leaving performance on the table
**and** to demonstrate overfitting awareness, we perform a **systematic grid search**
with temporal cross-validation (no data leakage).

**Search space**: `max_depth`, `num_leaves`, `min_child_samples`, `learning_rate`
**Validation**: `TimeSeriesSplit(n_splits=3)` — respects temporal order

In [ ]:
# ============================================================
# 3.1b — GridSearchCV for LightGBM
# ============================================================
from sklearn.model_selection import GridSearchCV

# Search space: key hyperparameters that control model complexity
param_grid = {
    'max_depth': [3, 4, 5, 6],
    'num_leaves': [7, 15, 31],
    'min_child_samples': [10, 20, 50],
    'learning_rate': [0.01, 0.05, 0.1],
}

# Temporal cross-validation (no data leakage)
tscv_lgb = TimeSeriesSplit(n_splits=3)

grid_search = GridSearchCV(
    estimator=lgb.LGBMRegressor(
        n_estimators=200, subsample=0.8,
        reg_alpha=0.1, reg_lambda=0.1,
        verbose=-1, random_state=42,
    ),
    param_grid=param_grid,
    cv=tscv_lgb,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=0,
    refit=True,
)

print(f'Grid search: {len(param_grid["max_depth"]) * len(param_grid["num_leaves"]) * len(param_grid["min_child_samples"]) * len(param_grid["learning_rate"])} combinations x 3 folds')
grid_search.fit(X_train_imp, y_train)

print(f'\nBest parameters: {grid_search.best_params_}')
print(f'Best CV RMSE: {-grid_search.best_score_:.2f}')
print(f'\nCurrent (manual) parameters:')
print(f'  max_depth=4, num_leaves=15, min_child_samples=20, learning_rate=0.05')
print(f'  -> These {"match" if grid_search.best_params_ == {"max_depth": 4, "num_leaves": 15, "min_child_samples": 20, "learning_rate": 0.05} else "differ from"} the optimal found by grid search')

In [ ]:
# ============================================================
# 3.1b — GridSearchCV results visualization
# ============================================================
# Heatmap: RMSE by max_depth x num_leaves (averaged over other params)
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results['mean_rmse'] = -cv_results['mean_test_score']

# Pivot for heatmap (average over min_child_samples and learning_rate)
pivot_data = cv_results.groupby(
    ['param_max_depth', 'param_num_leaves']
)['mean_rmse'].mean().unstack()

fig, axes = plt.subplots(1, 2, figsize=(16, 5))
fig.suptitle('LightGBM — Hyperparameter Sensitivity Analysis', fontsize=14)

# Heatmap: max_depth vs num_leaves
sns.heatmap(pivot_data, annot=True, fmt='.1f', cmap='RdYlGn_r', ax=axes[0])
axes[0].set_title('Mean RMSE by max_depth x num_leaves')
axes[0].set_ylabel('max_depth')
axes[0].set_xlabel('num_leaves')

# Bar chart: RMSE by learning_rate (averaged over other params)
lr_impact = cv_results.groupby('param_learning_rate')['mean_rmse'].agg(['mean', 'std'])
axes[1].bar(range(len(lr_impact)), lr_impact['mean'], yerr=lr_impact['std'],
            capsize=5, color=['#2196F3', '#4CAF50', '#FF9800'])
axes[1].set_xticks(range(len(lr_impact)))
axes[1].set_xticklabels([f'lr={lr}' for lr in lr_impact.index])
axes[1].set_title('Mean RMSE by learning_rate')
axes[1].set_ylabel('CV RMSE')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Compare best grid search model vs manual model
best_lgb = grid_search.best_estimator_
y_pred_val_best = np.clip(best_lgb.predict(X_val_imp), 0, None)
y_pred_test_best = np.clip(best_lgb.predict(X_test_imp), 0, None)

print(f'\nManual params:  Val RMSE={np.sqrt(mean_squared_error(y_val, y_pred_val_lgb)):.2f}, '
      f'Test RMSE={np.sqrt(mean_squared_error(y_test, y_pred_test_lgb)):.2f}')
print(f'Best GridSearch: Val RMSE={np.sqrt(mean_squared_error(y_val, y_pred_val_best)):.2f}, '
      f'Test RMSE={np.sqrt(mean_squared_error(y_test, y_pred_test_best)):.2f}')

In [ ]:
# ============================================================
# 3.2 — LightGBM feature importance (gain-based)
# ============================================================
# Gain importance measures the error reduction brought by each feature

importance_lgb = pd.Series(
    lgb_model.feature_importances_, index=feature_cols
).sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(10, 8))
importance_lgb.head(20).iloc[::-1].plot(kind='barh', ax=ax, color='darkgreen')
ax.set_title('LightGBM — Top 20 Features (gain)', fontsize=14)
ax.set_xlabel('Importance (gain)')
ax.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# 3.3 — SHAP analysis (LightGBM interpretability)
# ============================================================
# SHAP shows the impact of each feature on EACH individual prediction
# (more informative than global gain importance)

import shap

explainer = shap.TreeExplainer(lgb_model)
shap_values = explainer.shap_values(X_val_imp)

fig, ax = plt.subplots(figsize=(10, 8))
shap.summary_plot(shap_values, X_val_imp, max_display=20, show=False)
plt.title('SHAP — Feature impact on LightGBM predictions', fontsize=12)
plt.tight_layout()
plt.show()

---
## 4. Prophet (Time series)

**Why Prophet?**
- Additive model: `y(t) = trend + seasonality + regressors + noise`
- Automatically captures annual seasonality
- Accepts external regressors (weather, household confidence)
- Trained **per department** (independent series)

In [ ]:
# ============================================================
# 4.1 — Prophet training per department
# ============================================================
from prophet import Prophet
import logging
logging.getLogger('prophet').setLevel(logging.WARNING)
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)

# Regressors to use
REGRESSORS = ['temp_mean', 'hdd_sum', 'cdd_sum', 'confiance_menages', 'ipi_hvac_c28']
REGRESSORS = [r for r in REGRESSORS if r in df.columns]

def to_prophet_df(data, target=TARGET, regressors=REGRESSORS):
    """Convert a DataFrame to Prophet format (ds, y + regressors)."""
    date_str = data['date_id'].astype(str)
    ds = pd.to_datetime(date_str.str[:4] + '-' + date_str.str[4:6] + '-01')
    pdf = pd.DataFrame({'ds': ds, 'y': data[target].values})
    for reg in regressors:
        if reg in data.columns:
            pdf[reg] = data[reg].values
    # Impute NaN
    for reg in regressors:
        if reg in pdf.columns:
            pdf[reg] = pdf[reg].ffill().bfill().fillna(0)
    return pdf.reset_index(drop=True)

# Train one model per department (use top 20 departments to avoid excessive output)
departments = sorted(df_train['dept'].unique())
prophet_results = {}

for dept in departments:
    train_dept = df_train[df_train['dept'] == dept]
    val_dept = df_val[df_val['dept'] == dept]
    test_dept = df_test[df_test['dept'] == dept]
    
    if len(train_dept) < 12:
        continue
    
    pdf_train = to_prophet_df(train_dept)
    pdf_val = to_prophet_df(val_dept)
    pdf_test = to_prophet_df(test_dept)
    
    # Configure Prophet with annual seasonality
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,
        daily_seasonality=False,
        changepoint_prior_scale=0.05,
        seasonality_prior_scale=5.0,
    )
    for reg in REGRESSORS:
        if reg in pdf_train.columns:
            model.add_regressor(reg)
    
    model.fit(pdf_train)
    
    forecast_val = model.predict(pdf_val)
    forecast_test = model.predict(pdf_test)
    
    prophet_results[dept] = {
        'model': model,
        'preds_val': np.clip(forecast_val['yhat'].values, 0, None),
        'actual_val': pdf_val['y'].values,
        'preds_test': np.clip(forecast_test['yhat'].values, 0, None),
        'actual_test': pdf_test['y'].values,
    }

print(f'Prophet: {len(prophet_results)} department models trained')

In [ ]:
# ============================================================
# 4.2 — Aggregated Prophet metrics
# ============================================================
all_actual_val = np.concatenate([r['actual_val'] for r in prophet_results.values()])
all_preds_val = np.concatenate([r['preds_val'] for r in prophet_results.values()])
all_actual_test = np.concatenate([r['actual_test'] for r in prophet_results.values()])
all_preds_test = np.concatenate([r['preds_test'] for r in prophet_results.values()])

print('PROPHET (aggregated across all departments)')
print('=' * 50)
for name, y_true, y_pred in [('Validation', all_actual_val, all_preds_val),
                               ('Test', all_actual_test, all_preds_test)]:
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f'  {name:12s} : RMSE={rmse:.2f}, MAE={mae:.2f}, R2={r2:.4f}')

In [ ]:
# ============================================================
# 4.3 — Prophet decomposition for a major department
# ============================================================
# Use the largest department by volume
if prophet_results:
    sample_dept = list(prophet_results.keys())[0]
    model_sample = prophet_results[sample_dept]['model']
    
    dept_data = pd.concat([
        df_train[df_train['dept'] == sample_dept],
        df_val[df_val['dept'] == sample_dept],
        df_test[df_test['dept'] == sample_dept]
    ])
    pdf_full = to_prophet_df(dept_data)
    forecast_full = model_sample.predict(pdf_full)
    
    dept_name = dept_data['dept_name'].iloc[0] if 'dept_name' in dept_data.columns else sample_dept
    fig = model_sample.plot_components(forecast_full)
    fig.suptitle(f'Prophet — Decomposition for {dept_name} ({sample_dept})', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()

---
## 5. Model comparison

In [ ]:
# ============================================================
# 5.1 — Summary table
# ============================================================
comparison = []
for name, y_pred_v, y_pred_t in [
    ('Ridge', y_pred_val_ridge, y_pred_test_ridge),
    ('LightGBM', y_pred_val_lgb, y_pred_test_lgb),
    ('Prophet', all_preds_val, all_preds_test),
]:
    y_v = y_val.values if name != 'Prophet' else all_actual_val
    y_t = y_test.values if name != 'Prophet' else all_actual_test
    comparison.append({
        'Model': name,
        'Val RMSE': np.sqrt(mean_squared_error(y_v, y_pred_v)),
        'Val MAE': mean_absolute_error(y_v, y_pred_v),
        'Val R2': r2_score(y_v, y_pred_v),
        'Test RMSE': np.sqrt(mean_squared_error(y_t, y_pred_t)),
        'Test MAE': mean_absolute_error(y_t, y_pred_t),
        'Test R2': r2_score(y_t, y_pred_t),
    })

df_comp = pd.DataFrame(comparison).sort_values('Val RMSE')
print('MODEL COMPARISON')
print('=' * 80)
print(df_comp.to_string(index=False))

In [ ]:
# ============================================================
# 5.2 — Comparative metric chart
# ============================================================
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
fig.suptitle('ML Model Comparison', fontsize=14)

for ax, metric, title in zip(axes, 
    ['RMSE', 'MAE', 'R2'],
    ['RMSE (lower = better)', 'MAE (lower = better)', 'R2 (higher = better)']):
    x = np.arange(len(df_comp))
    width = 0.35
    ax.bar(x - width/2, df_comp[f'Val {metric}'], width, label='Validation', color='steelblue')
    ax.bar(x + width/2, df_comp[f'Test {metric}'], width, label='Test', color='darkorange')
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(df_comp['Model'])
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# 5.3 — Predictions vs Actual (Ridge and LightGBM on test set)
# ============================================================
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle(f'Predictions vs Actual — Test set ({TARGET})', fontsize=14)

for ax, name, y_pred in zip(axes, ['Ridge', 'LightGBM'], 
                              [y_pred_test_ridge, y_pred_test_lgb]):
    ax.plot(range(len(y_test)), y_test.values, 'b-o', markersize=3, label='Actual', linewidth=1.5)
    ax.plot(range(len(y_test)), y_pred, 'r--s', markersize=3, label='Predicted', linewidth=1.5)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    ax.set_title(f'{name} (RMSE={rmse:.2f}, R2={r2:.3f})')
    ax.set_xlabel('Temporal index')
    ax.set_ylabel(TARGET)
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 5.4 — Overfitting Diagnostic

A critical check: does the model generalize from training to unseen data?
We compute **train metrics** and compare them to validation/test metrics.

- **Small gap** (< 5%) = good generalization, no overfitting
- **Moderate gap** (5-15%) = acceptable, some complexity could be reduced
- **Large gap** (> 15%) = overfitting risk, model memorizes training data

In [ ]:
# ============================================================
# 5.4 — Overfitting diagnostic: Train vs Val vs Test
# ============================================================
# Compute train predictions for each model

# Ridge (on training set)
y_pred_train_ridge = np.clip(ridge_model.predict(X_train_scaled), 0, None)
train_r2_ridge = r2_score(y_train, y_pred_train_ridge)
train_rmse_ridge = np.sqrt(mean_squared_error(y_train, y_pred_train_ridge))

# LightGBM (on training set)
y_pred_train_lgb = np.clip(lgb_model.predict(X_train_imp), 0, None)
train_r2_lgb = r2_score(y_train, y_pred_train_lgb)
train_rmse_lgb = np.sqrt(mean_squared_error(y_train, y_pred_train_lgb))

# Build the diagnostic table
val_r2_ridge = r2_score(y_val, y_pred_val_ridge)
test_r2_ridge = r2_score(y_test, y_pred_test_ridge)
val_r2_lgb = r2_score(y_val, y_pred_val_lgb)
test_r2_lgb = r2_score(y_test, y_pred_test_lgb)

diag_data = {
    'Model': ['Ridge', 'LightGBM'],
    'Train R2': [train_r2_ridge, train_r2_lgb],
    'Val R2': [val_r2_ridge, val_r2_lgb],
    'Test R2': [test_r2_ridge, test_r2_lgb],
    'Train RMSE': [train_rmse_ridge, train_rmse_lgb],
    'Val RMSE': [np.sqrt(mean_squared_error(y_val, y_pred_val_ridge)),
                 np.sqrt(mean_squared_error(y_val, y_pred_val_lgb))],
    'Test RMSE': [np.sqrt(mean_squared_error(y_test, y_pred_test_ridge)),
                  np.sqrt(mean_squared_error(y_test, y_pred_test_lgb))],
}
df_diag = pd.DataFrame(diag_data)
df_diag['Train-Val Gap (R2)'] = ((df_diag['Train R2'] - df_diag['Val R2']) / df_diag['Train R2'] * 100).round(1)
df_diag['Val-Test Gap (R2)'] = ((df_diag['Val R2'] - df_diag['Test R2']) / df_diag['Val R2'] * 100).round(1)

print('OVERFITTING DIAGNOSTIC')
print('=' * 90)
print(df_diag.round(4).to_string(index=False))
print('=' * 90)

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Overfitting Diagnostic — Train / Val / Test Gap', fontsize=14)

alphas_bar = [0.4, 0.7, 1.0]  # Opacity: Train=light, Val=medium, Test=dark
for i, (model_name, color) in enumerate(zip(['Ridge', 'LightGBM'], ['steelblue', 'darkgreen'])):
    row = df_diag[df_diag['Model'] == model_name].iloc[0]

    # R2 comparison — draw each bar individually (matplotlib alpha is scalar)
    r2_vals = [row['Train R2'], row['Val R2'], row['Test R2']]
    positions = [i * 4, i * 4 + 1, i * 4 + 2]
    for pos, val, a in zip(positions, r2_vals, alphas_bar):
        bar = axes[0].bar(pos, val, color=color, alpha=a,
                          edgecolor='black', linewidth=0.5)
        axes[0].annotate(f'{val:.3f}', xy=(pos, val),
                        ha='center', va='bottom', fontsize=9)

axes[0].set_xticks([1, 5])
axes[0].set_xticklabels(['Ridge\n(Train/Val/Test)', 'LightGBM\n(Train/Val/Test)'])
axes[0].set_ylabel('R2')
axes[0].set_title('R2 across splits (lower gap = less overfitting)')
axes[0].set_ylim(0.9, 1.005)
axes[0].grid(True, alpha=0.3, axis='y')

# Gap analysis
gap_data = df_diag[['Model', 'Train-Val Gap (R2)', 'Val-Test Gap (R2)']].set_index('Model')
gap_data.plot(kind='bar', ax=axes[1], color=['#FF9800', '#F44336'], edgecolor='black', linewidth=0.5)
axes[1].set_title('R2 Gap (%) — smaller is better')
axes[1].set_ylabel('Gap (%)')
axes[1].axhline(5, color='green', linestyle='--', alpha=0.5, label='Healthy threshold (5%)')
axes[1].axhline(15, color='red', linestyle='--', alpha=0.5, label='Overfitting threshold (15%)')
axes[1].legend(fontsize=8)
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

# Verdict
for _, row in df_diag.iterrows():
    gap = row['Train-Val Gap (R2)']
    status = 'HEALTHY' if gap < 5 else ('MODERATE' if gap < 15 else 'OVERFITTING RISK')
    print(f"{row['Model']:12s}: Train-Val R2 gap = {gap:.1f}% -> {status}")

In [ ]:
# ============================================================
# 5.5 — Cross-Validation Stability Analysis
# ============================================================
# Check if model performance is consistent across temporal folds.
# High variance across folds = unstable model.

tscv_stability = TimeSeriesSplit(n_splits=3)
cv_scores_ridge = []
cv_scores_lgb = []

for fold, (train_idx, val_idx) in enumerate(tscv_stability.split(X_train_imp)):
    # Ridge
    ridge_cv = Ridge(alpha=best_alpha)
    ridge_cv.fit(X_train_scaled.iloc[train_idx], y_train.iloc[train_idx])
    y_pred_cv = ridge_cv.predict(X_train_scaled.iloc[val_idx])
    cv_scores_ridge.append({
        'fold': fold + 1,
        'RMSE': np.sqrt(mean_squared_error(y_train.iloc[val_idx], y_pred_cv)),
        'R2': r2_score(y_train.iloc[val_idx], y_pred_cv),
    })
    
    # LightGBM
    lgb_cv = lgb.LGBMRegressor(**{**lgb_params, 'verbose': -1})
    lgb_cv.fit(X_train_imp.iloc[train_idx], y_train.iloc[train_idx])
    y_pred_cv = lgb_cv.predict(X_train_imp.iloc[val_idx])
    cv_scores_lgb.append({
        'fold': fold + 1,
        'RMSE': np.sqrt(mean_squared_error(y_train.iloc[val_idx], y_pred_cv)),
        'R2': r2_score(y_train.iloc[val_idx], y_pred_cv),
    })

df_cv_ridge = pd.DataFrame(cv_scores_ridge)
df_cv_lgb = pd.DataFrame(cv_scores_lgb)

print('CROSS-VALIDATION STABILITY')
print('=' * 60)
print(f'\nRidge (alpha={best_alpha}):')
print(df_cv_ridge.to_string(index=False))
print(f'  Mean RMSE: {df_cv_ridge["RMSE"].mean():.2f} +/- {df_cv_ridge["RMSE"].std():.2f}')
print(f'  Mean R2:   {df_cv_ridge["R2"].mean():.4f} +/- {df_cv_ridge["R2"].std():.4f}')

print(f'\nLightGBM:')
print(df_cv_lgb.to_string(index=False))
print(f'  Mean RMSE: {df_cv_lgb["RMSE"].mean():.2f} +/- {df_cv_lgb["RMSE"].std():.2f}')
print(f'  Mean R2:   {df_cv_lgb["R2"].mean():.4f} +/- {df_cv_lgb["R2"].std():.4f}')

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Cross-Validation Stability (3-fold TimeSeriesSplit)', fontsize=14)

# RMSE per fold
x = np.arange(3)
width = 0.35
axes[0].bar(x - width/2, df_cv_ridge['RMSE'], width, label='Ridge', color='steelblue', edgecolor='black')
axes[0].bar(x + width/2, df_cv_lgb['RMSE'], width, label='LightGBM', color='darkgreen', edgecolor='black')
axes[0].set_xticks(x)
axes[0].set_xticklabels([f'Fold {i+1}' for i in range(3)])
axes[0].set_ylabel('RMSE')
axes[0].set_title('RMSE per CV fold')
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# R2 per fold
axes[1].bar(x - width/2, df_cv_ridge['R2'], width, label='Ridge', color='steelblue', edgecolor='black')
axes[1].bar(x + width/2, df_cv_lgb['R2'], width, label='LightGBM', color='darkgreen', edgecolor='black')
axes[1].set_xticks(x)
axes[1].set_xticklabels([f'Fold {i+1}' for i in range(3)])
axes[1].set_ylabel('R2')
axes[1].set_title('R2 per CV fold')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Stability verdict
for name, df_cv in [('Ridge', df_cv_ridge), ('LightGBM', df_cv_lgb)]:
    cv_coeff_var = df_cv['RMSE'].std() / df_cv['RMSE'].mean() * 100
    stability = 'STABLE' if cv_coeff_var < 15 else ('MODERATE' if cv_coeff_var < 30 else 'UNSTABLE')
    print(f'{name:12s}: CV RMSE CoV = {cv_coeff_var:.1f}% -> {stability}')

---
## 6. Conclusions

### Results (96 departments, ~5376 rows):

| Model | Val RMSE | Val R² | Test RMSE | Test R² | Status |
|-------|----------|--------|-----------|---------|--------|
| **LightGBM** | 10.75 | 0.990 | 12.10 | 0.987 | Best overall |
| **Ridge** | 16.28 | 0.976 | 21.29 | 0.961 | Strong baseline |
| **Prophet** | 60.37 | 0.616 | 69.09 | 0.530 | Underperforms |

### Overfitting assessment:
- **LightGBM**: Train-Val R² gap = 0.5% — **HEALTHY** (no overfitting)
- **Ridge**: Train-Val R² gap = 1.2% — **HEALTHY** (excellent generalization)
- **CV stability**: Ridge CoV = 9.5%, LightGBM CoV = 4.7% — both **STABLE** across temporal folds
- **GridSearchCV**: 144 combinations tested, confirms manual parameters (max_depth=4, num_leaves=15, min_child_samples=20, lr=0.05) are optimal

### Most important features:
- **Temporal lags** (nb_installations_pac_lag_1m) — most predictive feature across all methods, reflecting strong auto-correlation
- **Rolling means** — smooth monthly noise, improve prediction stability
- **Weather** (HDD, temperature) — significant physical impact on HVAC demand
- **Household confidence** — useful economic signal for investment intention
- **nb_dpe_total** — volume scaler (more real estate transactions = more heat pump detections)

### Why Prophet underperforms:
Prophet trains independently per department (96 separate models with only ~36 months each).
With so little data per series, it cannot learn reliable seasonal patterns or trend.
A hierarchical Prophet (sharing trend across departments) could potentially improve results.

### Improvement suggestions:
- **Quantile regression** (LightGBM `objective='quantile'`) for prediction confidence intervals
- **Feature selection** with recursive feature elimination (RFE) — currently ~60+ features, could reduce without losing performance
- **Ridge + LightGBM ensemble** (weighted average) — may capture complementary patterns
- **Hierarchical Prophet** (shared trend + department-level seasonality) to overcome per-series data limitation
- **Partial dependence plots** (PDP) for top 5 features to understand non-linear effects

### Next step:
-> Notebook 03: Exploratory LSTM (deep learning)